# Classification of Plant Leaf Diseases using MLPs (32x32 images)

## Imports

In [1]:
# General imports 
import sys
import os 
sys.path.insert(1, os.path.join(os.pardir, 'src'))

# Data imports
import cv2
import plotly.express as px
import numpy as np
import mlflow
import matplotlib.pyplot as plt

# Homebrew imports 
import model
from utils import one_hot_encode_index
from optimizers import Adam
from activations import Softmax, ReLU
from layers import Dropout, LinearLayer
from loss import CategoricalCrossEntropyLoss

## TESTING 
import importlib
importlib.reload(model)
##

<module 'model' from '..\\src\\model.py'>

## Loading & Preprocessing Data 

In [4]:
data_path = os.path.join(os.pardir, 'data', 'Plant_leave_diseases_32')

X = []
y = []

for _class in os.listdir(data_path):
    for img_name in os.listdir(os.path.join(data_path, _class)):
        img = cv2.imread(os.path.join(data_path, _class, img_name), cv2.IMREAD_GRAYSCALE)
        X.append(img)
        y.append(_class)

X = np.array(X)

# Scaling X between -1, 1  
X = (X - np.mean(X))/ np.mean(X)

# Vectorizing X
X = X.reshape(X.shape[0], -1)

# encoding y
class_lbl_to_int = dict([(y, i) for i, y in enumerate(sorted(set(y)))])
y = np.array([class_lbl_to_int[_y] for _y in y]).astype(np.uint8)
y = one_hot_encode_index(y, len(class_lbl_to_int))


## Shuffling & splitting data

Since the data is large we will use a random 90,5,5 split for training, validation and testing respectively 

In [5]:
n_samples = X.shape[0]
keys = np.array(range(X.shape[0]))
np.random.shuffle(keys)

train_keys = keys[:int(n_samples*0.9)]
validation_keys = keys[int(n_samples*0.9):int(n_samples*0.9 + n_samples*0.05)]
test_keys = keys[int(n_samples*0.9 + n_samples*0.05):]

X_train = X[train_keys]
X_val = X[validation_keys]
X_test = X[test_keys]

y_train = y[train_keys]
y_val = y[validation_keys]
y_test = y[test_keys]

## Good modls 

In [6]:
# initialize model 
mdl = model.Model(Adam(),CategoricalCrossEntropyLoss())

# Config early stop 
mdl.add_early_stop(5)

# Defining architecture 
mdl.set_sequence([
                LinearLayer(X.shape[1], 800),
                ReLU(),
                LinearLayer(800, 500), 
                ReLU(),
                LinearLayer(500, 50),  
                ReLU(),
                LinearLayer(50, len(class_lbl_to_int)),
                Softmax()
            ])

In [7]:
mlflow.set_experiment(experiment_name='Node test')
with mlflow.start_run():
        # Training 
        mlflow.log_param('img_size', '32x32')
        mdl.train(X_train, y_train, epochs=500, batch_size=64, validation=(X_test, y_test))

INFO: 'Node test' does not exist. Creating a new experiment
=== Epoch: 1 ===
Step: 0/865, accuracy0.000, loss3.679, learning rate 0.0010000 
Step: 100/865, accuracy0.281, loss2.443, learning rate 0.0010000 
Step: 200/865, accuracy0.250, loss2.444, learning rate 0.0010000 
Step: 300/865, accuracy0.344, loss2.259, learning rate 0.0010000 
Step: 400/865, accuracy0.531, loss1.778, learning rate 0.0010000 
Step: 500/865, accuracy0.516, loss1.839, learning rate 0.0010000 
Step: 600/865, accuracy0.469, loss1.978, learning rate 0.0010000 
Step: 700/865, accuracy0.438, loss1.853, learning rate 0.0010000 
Step: 800/865, accuracy0.469, loss1.811, learning rate 0.0010000 
Step: 864/865, accuracy0.634, loss1.454, learning rate 0.0010000 
Epoch: 1/500, accuracy0.400, loss2.199, learning rate 0.001
Estimated reamining runtime: 7:04:43.929157
--Validation--
Validation : Accuracy: 0.484, Loss: 1.863
=== Epoch: 2 ===
Step: 0/865, accuracy0.531, loss1.605, learning rate 0.0010000 
Step: 100/865, accuracy

In [16]:
# initialize model 
mdl = model.Model(Adam(learning_rate=0.002, decay=0.001),CategoricalCrossEntropyLoss())

# Config early stop 
mdl.add_early_stop(5)

# Defining architecture 
mdl.set_sequence([
                LinearLayer(X.shape[1], 1024),
                ReLU(),
                Dropout(0.5),
                LinearLayer(1024, 512), 
                ReLU(),
                Dropout(0.5),
                LinearLayer(512, len(class_lbl_to_int)),
                Softmax()
            ])

In [17]:
mlflow.set_experiment(experiment_name='Node test')
with mlflow.start_run():
        # Training 
        mlflow.log_param('img_size', '32x32')
        mdl.train(X_train, y_train, epochs=500, batch_size=64, validation=(X_test, y_test))

=== Epoch: 1 ===
Step: 0/865, accuracy0.031, loss4.029, learning rate 0.0020000 
Step: 100/865, accuracy0.250, loss2.894, learning rate 0.0018182 
Step: 200/865, accuracy0.219, loss2.684, learning rate 0.0016667 
Step: 300/865, accuracy0.281, loss2.571, learning rate 0.0015385 
Step: 400/865, accuracy0.328, loss2.389, learning rate 0.0014286 
Step: 500/865, accuracy0.344, loss2.324, learning rate 0.0013333 
Step: 600/865, accuracy0.344, loss2.492, learning rate 0.0012500 
Step: 700/865, accuracy0.344, loss2.239, learning rate 0.0011765 
Step: 800/865, accuracy0.359, loss2.267, learning rate 0.0011111 
Step: 864/865, accuracy0.415, loss2.255, learning rate 0.0010730 
Epoch: 1/500, accuracy0.306, loss2.596, learning rate 0.001
Estimated reamining runtime: 9:28:19.992037
--Validation--
Validation : Accuracy: 0.430, Loss: 2.046
=== Epoch: 2 ===
Step: 0/865, accuracy0.391, loss2.041, learning rate 0.0010724 
Step: 100/865, accuracy0.406, loss2.046, learning rate 0.0010178 
Step: 200/865, ac

In [2]:
def load_set(pth):
    X = []
    y = []

    for _class in os.listdir(pth):
        for img_name in os.listdir(os.path.join(pth, _class)):
            img = cv2.imread(os.path.join(pth, _class, img_name), cv2.IMREAD_GRAYSCALE)
            X.append(img)
            y.append(_class)

    X = np.array(X)

    # Normalizing 
    X = (X - np.mean(X))/ np.mean(X)

    # Vectorizing X
    X = X.reshape(X.shape[0], -1)

    # encoding y
    class_lbl_to_int = dict([(y, i) for i, y in enumerate(sorted(set(y)))])
    y = np.array([class_lbl_to_int[_y] for _y in y]).astype(np.uint8)
    y = one_hot_encode_index(y, len(class_lbl_to_int))

    return X, y


def load_plant_data(resolution=32):
    base_dir = os.path.join(os.pardir, 'data', f'Plant_leave_diseases_{resolution}')
    train_dir = os.path.join(base_dir, 'train')
    validation_dir = os.path.join(base_dir, 'validation')

    # loading train data 
    X_train, y_train = load_set(train_dir)

    # loading validation data 
    X_validation, y_validation = load_set(validation_dir)

    return X_train, y_train, X_validation, y_validation

In [3]:
X_train, y_train, X_validation, y_validation = load_plant_data()


In [6]:
np.argmax(y_train, axis=1)

array([ 0,  0,  0, ..., 36, 36, 36], dtype=int64)